# 文本向量化构建索引的过程

### 1. MIssing Content(内容缺失): 原本的文本中就没有问题的答案


增加相应知识库：将相应的知识文本加入到向量知识库中。
数据清洗与增强：输入垃圾，那也必定输出垃圾。任何RAG工作流程想要获得优良表现，都必须先清洁数据。
更好的Prompt设计：通过Prompts，比如让大模型在找不到答案的情况下，输出“根据当前知识库，无法回答该问题”等提示。

### 2. 文档加载准确性和效率： 比如pdf文件的加载，如何提取其中的有用文字信息和图片信息等

优化文档读取器：一般知识库中的文档格式都不尽相同，针对每一类文档，涉及一个专门的读取器。(可能需要自定义读取器)
数据清洗与增强

### 3. 文档切分的粒度： 文本切分的大小和位置会影响后面检索出来的上下文完整性和与大模型交互的token数量，怎么控制好文档切分的度，是个难题。

内容重叠分块：为了保持文本块之间语义上下文的连贯性，在分块时，保持文本块之间有一定的内容重叠。

基于结构的分块：基于结构的分块方法利用文档的固有结构，如HTML或Markdown中的标题和段落，以保持内容的逻辑性和完整性。

基于递归的分块：重复的利用分块规则不断细分文本块。比如先通过段落换行符（\n\n）进行分割。然后，检查这些块的大小。如果大小不超过一定阈值，则该块被保留。对于大小超过标准的块，使用单换行符（\n）再次分割。以此类推，不断根据块大小更新更小的分块规则（如空格，句号）。

分块大小的选择：  
	不同的嵌入模型有其最佳输入大小。比如Openai的text-embedding-ada-002的模型在256 或 512大小的块上效果更好。  
	文档的类型和用户查询的长度及复杂性也是决定分块大小的重要因素。


# 检索增强回答的过程中

### 1. Missed Top Ranked: 错过排名靠前的文档

外挂知识库中存在回答问题所需的知识，但是可能这个知识块与问题的向量相似度排名并不是靠前的，导致无法召回该知识块传给大模型，导致大模型始终无法得到正确的答案。 
- 增加召回数量：增加召回的 topK 数量，也就是说，例如原来召回前3个知识块，修改为召回前5个知识块。不过此种方法，因为知识块多了，不光会增加token消耗，也会增加大模型回答问题的干扰。
- 重排（Reranking）利用重排序模型进行重新排序
- 提高向量相似度

### 2. Not in Context:  提取上下文与答案无关

内容缺失 或 错过排名靠前的文档 的具体体现

### 3. Wrong Format(格式错误): 例如需要Json，给了字符串

- Prompt调优优化Prompt逐渐让大模型返回正确的格式。
- 进行结果格式验证，例如使用LangChain中的PydanticOutputParser类来校验输出格式。
- Auto-Fixing自修复：对不符合要求的格式进行自动修复

### 4. Incomplete(答案不完整): 答案只回答了问题的一部分
#### 大模型能力不足，只回答了部分问题
#### 知识库检索内容只有部分

- 将问题分开提问一方面引导用户精简问题，一次只提问一个问题。 另一方面，针对用户的问题进行内部拆分处理，拆分成数个子问题，等子问题答案都找到后，再总结起来回复给用户

### 5. Not Extracted(未提取到答案:) 提取的上下文中有答案，但大模型没有提取出来

- 提示压缩技术

### 6. Incorrect Specificity: 答案不够具体或过于具体

- 提示词改善 或者 提升基座大模型能力